<a href="https://colab.research.google.com/github/Monunep/DPDPD/blob/main/DPDPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import annotations
import math
import random
import heapq
import copy
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Any, Set

import numpy as np

NODE_TYPE, TRUCK_DIST, DRONE_DIST, TRUCK_TIME, DRONE_TIME = 0, 1, 2, 3, 4
TYPE_DEPOT, TYPE_HUB, TYPE_CUSTOMER = 1, 2, 3

NUM_TRUCKS = 3
TRUCK_CAPACITY = 100
TRUCK_SPEED = 1.0
DRONES_PER_HUB = 2
DRONES_PER_TRUCK = 1
DRONE_CAPACITY = 10
DRONE_SPEED = 10.0
DRONE_MAX_RANGE_DIST = 150
SERVICE_TIME = 5

PENALTY_UNSERVED = 10000
PENALTY_TARDINESS = 100

ALNS_ITERATIONS = 1000
COOLING_RATE = 0.995
START_TEMPERATURE = 500.0
END_TEMPERATURE = 1.0
REHEAT_IF_NO_IMPROVE_ITERATIONS = 400
REHEAT_FACTOR = 1.5
DESTROY_PERCENTAGE = 0.25
SHAW_REMOVAL_SIZE = 6
REGRET_K = 2
WEIGHT_UPDATE_REACTION_FACTOR = 0.3

def create_random_instance_data(num_nodes: int, num_hubs: int, num_customers: int,
                                max_dist: float = 100, truck_speed: float = 1.0,
                                drone_speed: float = 2.0) -> Tuple[np.ndarray, List[Dict]]:
    if 1 + num_hubs + num_customers != num_nodes:
        raise ValueError("Tổng số hubs và customers phải bằng num_nodes - 1.")

    A = np.zeros((num_nodes, num_nodes, 5))
    node_properties = []

    node_ids = list(range(1, num_nodes))
    random.shuffle(node_ids)

    A[0, 0, NODE_TYPE] = TYPE_DEPOT
    node_properties.append({'id': 0, 'type': TYPE_DEPOT, 'demand': 0, 'ready_time': 0, 'due_date': 2000})

    for _ in range(num_hubs):
        hub_id = node_ids.pop()
        A[hub_id, hub_id, NODE_TYPE] = TYPE_HUB
        node_properties.append({'id': hub_id, 'type': TYPE_HUB, 'demand': 0, 'ready_time': 0, 'due_date': float('inf')})

    for cust_id in node_ids:
        A[cust_id, c ust_id, NODE_TYPE] = TYPE_CUSTOMER
        demand = random.randint(5, 15)
        ready_time = random.randint(0, 800)
        due_date = ready_time + random.randint(100, 300)
        node_properties.append({'id': cust_id, 'type': TYPE_CUSTOMER, 'demand': demand,
                                'ready_time': ready_time, 'due_date': due_date})

    node_properties.sort(key=lambda x: x['id'])

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue
            disconnect = random.random()
            dist = random.uniform(5, max_dist)
            if disconnect > 0.1:
                A[i, j, TRUCK_DIST] = dist
                A[i, j, DRONE_DIST] = dist
                A[i, j, TRUCK_TIME] = dist / truck_speed
                A[i, j, DRONE_TIME] = dist / drone_speed
            else:
                A[i, j, TRUCK_DIST] = 1000000
                A[i, j, DRONE_DIST] = 1000000
                A[i, j, TRUCK_TIME] = 1000000 / truck_speed
                A[i, j, DRONE_TIME] = 1000000 / drone_speed

    return A, node_properties

@dataclass
class Instance:
    adjacency_matrix: np.ndarray
    node_properties_list: List[Dict[str, Any]]
    depot_id: int = 0
    truck_time_matrix: np.ndarray = field(init=False)
    truck_dist_matrix: np.ndarray = field(init=False)
    drone_time_matrix: np.ndarray = field(init=False)
    drone_dist_matrix: np.ndarray = field(init=False)
    node_types: np.ndarray = field(init=False)
    nodes: Dict[int, Dict[str, Any]] = field(init=False)
    hub_ids: List[int] = field(init=False)
    customer_ids: List[int] = field(init=False)

    def __post_init__(self):
        self.truck_time_matrix = self.adjacency_matrix[:, :, TRUCK_TIME]
        self.truck_dist_matrix = self.adjacency_matrix[:, :, TRUCK_DIST]
        self.drone_time_matrix = self.adjacency_matrix[:, :, DRONE_TIME]
        self.drone_dist_matrix = self.adjacency_matrix[:, :, DRONE_DIST]
        self.node_types = self.adjacency_matrix[:, :, NODE_TYPE].diagonal().astype(int)
        self.nodes = {n['id']: n for n in self.node_properties_list}
        self.hub_ids = [i for i, t in enumerate(self.node_types) if t == TYPE_HUB]
        self.customer_ids = [i for i, t in enumerate(self.node_types) if t == TYPE_CUSTOMER]

@dataclass
class Solution:
    truck_routes: Dict[str, List[int]]
    hub_drone_assignments: Dict[int, List[int]]
    truck_launched_assignments: List[Dict[str, Any]]
    unserved_requests: List[int]

    def copy(self) -> 'Solution':
        return Solution(
            truck_routes={k: v.copy() for k, v in self.truck_routes.items()},
            hub_drone_assignments={k: v.copy() for k, v in self.hub_drone_assignments.items()},
            truck_launched_assignments=[a.copy() for a in self.truck_launched_assignments],
            unserved_requests=self.unserved_requests.copy(),
        )

@dataclass
class SimulationResult:
    total_cost: float
    is_feasible: bool
    tardiness_cost: float
    unserved_cost: float
    late_customer_count: int
    unserved_customer_count: int
    infeasibility_reason: Optional[str] = None

class Simulator:
    def __init__(self, instance: Instance):
        self.instance = instance

    def evaluate(self, solution: Solution) -> SimulationResult:
        all_assignments = []
        for route in solution.truck_routes.values():
            all_assignments.extend(route)
        for reqs in solution.hub_drone_assignments.values():
            all_assignments.extend(reqs)
        for mission in solution.truck_launched_assignments:
            all_assignments.append(mission['req_id'])
        if len(set(all_assignments)) != len(all_assignments):
            return SimulationResult(float('inf'), False, 0, 0, 0, 0, 'Gán trùng lặp khách hàng')

        truck_eval = self._evaluate_truck_routes(solution)
        if not truck_eval['is_feasible']:
            return SimulationResult(float('inf'), False, 0, 0, 0, 0, truck_eval['reason'])

        hub_eval = self._evaluate_hub_missions(solution, truck_eval['hub_arrival_times'])
        if not hub_eval['is_feasible']:
            return SimulationResult(float('inf'), False, 0, 0, 0, 0, hub_eval['reason'])

        penalty_eval = self._calculate_final_penalties(solution, truck_eval['served_customers'] | hub_eval['served_customers'])

        total_cost = (truck_eval['base_cost'] + hub_eval['base_cost'] +
                      truck_eval['tardiness_cost'] + hub_eval['tardiness_cost'] +
                      penalty_eval['unserved_cost'])

        return SimulationResult(
            total_cost=total_cost,
            is_feasible=True,
            tardiness_cost=truck_eval['tardiness_cost'] + hub_eval['tardiness_cost'],
            unserved_cost=penalty_eval['unserved_cost'],
            late_customer_count=len(truck_eval['late_customers'] | hub_eval['late_customers']),
            unserved_customer_count=penalty_eval['unserved_count']
        )

    def _evaluate_truck_routes(self, solution: Solution) -> Dict:
        I = self.instance
        result = {'is_feasible': True, 'reason': '', 'base_cost': 0.0, 'tardiness_cost': 0.0,
                  'late_customers': set(), 'served_customers': set(), 'hub_arrival_times': {}}

        for truck_id, route in solution.truck_routes.items():
            payload = sum(I.nodes[c]['demand'] for c in route)
            payload += sum(I.nodes[m['req_id']]['demand'] for m in solution.truck_launched_assignments if m['truck_id'] == truck_id)
            if payload > TRUCK_CAPACITY:
                result['is_feasible'], result['reason'] = False, f'Tải trọng xe {truck_id} bị vượt'
                return result

            current_time = 0.0
            sequence = [I.depot_id] + route + [I.depot_id]
            for i in range(len(sequence) - 1):
                start_node, end_node = sequence[i], sequence[i + 1]
                departure_time = current_time
                result['base_cost'] += I.truck_dist_matrix[start_node, end_node]

                missions = [m for m in solution.truck_launched_assignments if m['truck_id'] == truck_id and m['launch'] == start_node and m['rendezvous'] == end_node]
                if len(missions) > DRONES_PER_TRUCK:
                    result['is_feasible'], result['reason'] = False, 'Vượt quá số lượng drone trên xe'
                    return result

                for mission in missions:
                    cust_id = mission['req_id']
                    flight_time = I.drone_time_matrix[start_node, cust_id] + SERVICE_TIME + I.drone_time_matrix[cust_id, end_node]
                    if flight_time > I.truck_time_matrix[start_node, end_node]:
                        result['is_feasible'], result['reason'] = False, 'Lỗi đồng bộ hóa drone'
                        return result

                    arrival_at_cust = departure_time + I.drone_time_matrix[start_node, cust_id]
                    start_service = max(arrival_at_cust, I.nodes[cust_id]['ready_time'])
                    if start_service > I.nodes[cust_id]['due_date']:
                        tardiness = start_service - I.nodes[cust_id]['due_date']
                        result['tardiness_cost'] += tardiness * PENALTY_TARDINESS
                        result['late_customers'].add(cust_id)

                    result['base_cost'] += I.drone_dist_matrix[start_node, cust_id] + I.drone_dist_matrix[cust_id, end_node]
                    result['served_customers'].add(cust_id)

                current_time += I.truck_time_matrix[start_node, end_node]
                current_time = max(current_time, I.nodes[end_node]['ready_time'])

                if I.node_types[end_node] == TYPE_CUSTOMER:
                    if current_time > I.nodes[end_node]['due_date']:
                        tardiness = current_time - I.nodes[end_node]['due_date']
                        result['tardiness_cost'] += tardiness * PENALTY_TARDINESS
                        result['late_customers'].add(end_node)
                    current_time += SERVICE_TIME
                    result['served_customers'].add(end_node)
                elif I.node_types[end_node] == TYPE_HUB:
                    if end_node not in result['hub_arrival_times']:
                        result['hub_arrival_times'][end_node] = current_time
        return result

    def _evaluate_hub_missions(self, solution: Solution, hub_arrival_times: Dict) -> Dict:
        I = self.instance
        result = {'is_feasible': True, 'reason': '', 'base_cost': 0.0, 'tardiness_cost': 0.0,
                  'late_customers': set(), 'served_customers': set()}

        for hub_id, requests in solution.hub_drone_assignments.items():
            if not requests:
                continue
            if hub_id not in hub_arrival_times:
                result['is_feasible'], result['reason'] = False, f'Trạm {hub_id} có nhiệm vụ nhưng không được xe ghé thăm'
                return result

            drone_availability_times = [hub_arrival_times[hub_id]] * DRONES_PER_HUB
            heapq.heapify(drone_availability_times)

            sorted_requests = sorted(requests, key=lambda c: I.nodes[c]['due_date'])

            for cust_id in sorted_requests:
                drone_ready_time = heapq.heappop(drone_availability_times)
                arrival_at_cust = drone_ready_time + I.drone_time_matrix[hub_id, cust_id]
                start_service = max(arrival_at_cust, I.nodes[cust_id]['ready_time'])

                if start_service > I.nodes[cust_id]['due_date']:
                    tardiness = start_service - I.nodes[cust_id]['due_date']
                    result['tardiness_cost'] += tardiness * PENALTY_TARDINESS
                    result['late_customers'].add(cust_id)

                finish_time = start_service + SERVICE_TIME + I.drone_time_matrix[cust_id, hub_id]
                heapq.heappush(drone_availability_times, finish_time)
                result['base_cost'] += 2 * I.drone_dist_matrix[hub_id, cust_id]
                result['served_customers'].add(cust_id)
        return result

    def _calculate_final_penalties(self, solution: Solution, served_customers: Set) -> Dict:
        all_customers = set(self.instance.customer_ids)
        declared_unserved = set(solution.unserved_requests)
        truly_unserved = all_customers - served_customers
        unserved_cost = len(truly_unserved) * PENALTY_UNSERVED
        return {'unserved_cost': unserved_cost, 'unserved_count': len(truly_unserved)}

class Operators:
    def __init__(self, instance: Instance, simulator: Simulator):
        self.instance = instance
        self.simulator = simulator

    def random_removal(self, solution: Solution, percentage: float = DESTROY_PERCENTAGE) -> Tuple[Solution, List[int]]:
        s = solution.copy()
        served = self._get_served_customer_list(s)
        if not served:
            return s, []
        num_to_remove = max(1, int(len(served) * percentage))
        to_remove = random.sample(served, num_to_remove)
        self._remove_requests_from_solution(s, to_remove)
        return s, to_remove

    def shaw_removal(self, solution: Solution, size: int = SHAW_REMOVAL_SIZE) -> Tuple[Solution, List[int]]:
        s = solution.copy()
        served = self._get_served_customer_list(s)
        if not served:
            return s, []
        seed_customer = random.choice(served)

        def similarity(c1, c2):
            dist = self.instance.truck_dist_matrix[c1, c2]
            r1, d1 = self.instance.nodes[c1]['ready_time'], self.instance.nodes[c1]['due_date']
            r2, d2 = self.instance.nodes[c2]['ready_time'], self.instance.nodes[c2]['due_date']
            overlap = max(0, min(d1, d2) - max(r1, r2))
            return 0.7 * dist + 0.3 * (1.0 - overlap / (d1 - r1 + d2 - r2 + 1e-6))

        ranked = sorted(served, key=lambda c: similarity(seed_customer, c))
        to_remove = ranked[:min(size, len(ranked))]
        self._remove_requests_from_solution(s, to_remove)
        return s, to_remove

    def regret_k_insertion(self, solution: Solution, requests: List[int]) -> Solution:
        s = solution.copy()
        while requests:
            best_customer_to_insert = None
            highest_regret = -float('inf')
            best_move_for_best_customer = None
            for cust_id in requests:
                candidate_moves = self._find_all_insertion_moves(s, cust_id)
                if not candidate_moves:
                    continue
                candidate_moves.sort(key=lambda x: x[0])
                best_cost = candidate_moves[0][0]
                regret = 0
                for i in range(1, min(REGRET_K, len(candidate_moves))):
                    regret += candidate_moves[i][0] - best_cost
                if regret > highest_regret:
                    highest_regret = regret
                    best_customer_to_insert = cust_id
                    best_move_for_best_customer = candidate_moves[0]
            if best_customer_to_insert:
                s = self._apply_move(s, best_customer_to_insert, best_move_for_best_customer)
                requests.remove(best_customer_to_insert)
            else:
                break
        return s

    def _find_all_insertion_moves(self, s, cust_id):
        moves = []
        for truck_id, route in s.truck_routes.items():
            for i in range(len(route) + 1):
                temp_sol = s.copy()
                temp_sol.truck_routes[truck_id].insert(i, cust_id)
                if cust_id in temp_sol.unserved_requests:
                    temp_sol.unserved_requests.remove(cust_id)
                res = self.simulator.evaluate(temp_sol)
                if res.is_feasible:
                    moves.append((res.total_cost, ('truck', truck_id, i)))

        for hub_id in self.instance.hub_ids:
            temp_sol = s.copy()
            if len(temp_sol.hub_drone_assignments.get(hub_id, [])) < DRONES_PER_HUB:
                temp_sol.hub_drone_assignments.setdefault(hub_id, []).append(cust_id)
                if cust_id in temp_sol.unserved_requests:
                    temp_sol.unserved_requests.remove(cust_id)
                res = self.simulator.evaluate(temp_sol)
                if res.is_feasible:
                    moves.append((res.total_cost, ('hub', hub_id)))

        for truck_id, route in s.truck_routes.items():
            seq = [self.instance.depot_id] + route + [self.instance.depot_id]
            current_truck_drone_missions = len([m for m in s.truck_launched_assignments if m['truck_id'] == truck_id])
            if current_truck_drone_missions < DRONES_PER_TRUCK:
                for i in range(len(seq) - 1):
                    launch_node, rendezvous_node = seq[i], seq[i + 1]
                    dist_to_cust = self.instance.drone_dist_matrix[launch_node, cust_id]
                    dist_from_cust = self.instance.drone_dist_matrix[cust_id, rendezvous_node]
                    if dist_to_cust + dist_from_cust <= DRONE_MAX_RANGE_DIST:
                        temp_sol = s.copy()
                        mission = {'req_id': cust_id, 'truck_id': truck_id,
                                   'launch': launch_node, 'rendezvous': rendezvous_node}
                        temp_sol.truck_launched_assignments.append(mission)
                        if cust_id in temp_sol.unserved_requests:
                            temp_sol.unserved_requests.remove(cust_id)
                        res = self.simulator.evaluate(temp_sol)
                        if res.is_feasible:
                            moves.append((res.total_cost, ('truck_launch', mission)))
        return moves

    def _apply_move(self, s: Solution, cust_id: int, move: Tuple) -> Solution:
        cost, details = move[0], move[1]
        move_type = details[0]
        if cust_id in s.unserved_requests:
            s.unserved_requests.remove(cust_id)
        if move_type == 'truck':
            truck_id, pos = details[1:]
            s.truck_routes[truck_id].insert(pos, cust_id)
        elif move_type == 'hub':
            hub_id = details[1]
            s.hub_drone_assignments.setdefault(hub_id, []).append(cust_id)
        elif move_type == 'truck_launch':
            mission = details[1]
            s.truck_launched_assignments.append(mission)
        return s

    def _remove_requests_from_solution(self, s: Solution, reqs: List[int]) -> None:
        for tid in s.truck_routes:
            s.truck_routes[tid] = [c for c in s.truck_routes[tid] if c not in reqs]
        for hid in s.hub_drone_assignments:
            s.hub_drone_assignments[hid] = [c for c in s.hub_drone_assignments[hid] if c not in reqs]
        s.truck_launched_assignments = [m for m in s.truck_launched_assignments if m['req_id'] not in reqs]
        s.unserved_requests = list(set(s.unserved_requests) | set(reqs))

    def _get_served_customer_list(self, s: Solution) -> List[int]:
        served = set()
        for r in s.truck_routes.values():
            served.update(r)
        for lst in s.hub_drone_assignments.values():
            served.update(lst)
        for a in s.truck_launched_assignments:
            served.add(a['req_id'])
        return [c for c in served if self.instance.node_types[c] == TYPE_CUSTOMER]

class ALNS:
    def __init__(self, instance: Instance):
        self.instance = instance
        self.simulator = Simulator(instance)
        self.operators = Operators(instance, self.simulator)
        self.destroy_operators = [self.operators.random_removal, self.operators.shaw_removal]
        self.repair_operators = [self.operators.regret_k_insertion]
        self.weights_destroy = [1.0] * len(self.destroy_operators)
        self.weights_repair = [1.0] * len(self.repair_operators)
        self.scores_destroy = [0.0] * len(self.destroy_operators)
        self.scores_repair = [0.0] * len(self.repair_operators)
        self.counts_destroy = [0] * len(self.destroy_operators)
        self.counts_repair = [0] * len(self.repair_operators)

    def run(self, initial_solution: Solution) -> Tuple[Solution, SimulationResult]:
        best_solution = initial_solution.copy()
        best_result = self.simulator.evaluate(best_solution)
        current_solution = initial_solution.copy()
        current_result = best_result

        temperature = START_TEMPERATURE
        iterations_since_last_improvement = 0

        print(f"Initial Cost: {best_result.total_cost:.2f} | Tardy: {best_result.late_customer_count}, Unserved: {best_result.unserved_customer_count}")
        if not best_result.is_feasible:
            print(f"Initial Solution is infeasible: {best_result.infeasibility_reason}")

        for iteration in range(1, ALNS_ITERATIONS + 1):
            destroy_op_idx = self._roulette_wheel_selection(self.weights_destroy)
            repair_op_idx = self._roulette_wheel_selection(self.weights_repair)
            destroy_operator = self.destroy_operators[destroy_op_idx]
            repair_operator = self.repair_operators[repair_op_idx]

            partial_solution, removed_requests = destroy_operator(current_solution)
            candidate_solution = repair_operator(partial_solution, removed_requests)
            candidate_result = self.simulator.evaluate(candidate_solution)

            score = 0
            if candidate_result.is_feasible and (not best_result.is_feasible or candidate_result.total_cost < best_result.total_cost):
                best_solution, best_result = candidate_solution.copy(), candidate_result
                current_solution, current_result = candidate_solution.copy(), candidate_result
                score = 3
                iterations_since_last_improvement = 0
                print(f"Iter {iteration} -> NEW BEST: {best_result.total_cost:.2f} (Tardy: {best_result.late_customer_count}, Unserved: {best_result.unserved_customer_count})")
            elif candidate_result.total_cost < current_result.total_cost:
                current_solution, current_result = candidate_solution.copy(), candidate_result
                score = 2
                iterations_since_last_improvement += 1
            else:
                acceptance_prob = math.exp((current_result.total_cost - candidate_result.total_cost) / max(1e-9, temperature))
                if random.random() < acceptance_prob:
                    current_solution, current_result = candidate_solution.copy(), candidate_result
                    score = 1
                iterations_since_last_improvement += 1

            self.scores_destroy[destroy_op_idx] += score
            self.counts_destroy[destroy_op_idx] += 1
            self.scores_repair[repair_op_idx] += score
            self.counts_repair[repair_op_idx] += 1
            if iteration % 100 == 0:
                self._update_weights()

            temperature = max(END_TEMPERATURE, temperature * COOLING_RATE)
            if iterations_since_last_improvement >= REHEAT_IF_NO_IMPROVE_ITERATIONS:
                temperature *= REHEAT_FACTOR
                iterations_since_last_improvement = 0

        print(f"\nFinished. Best cost found: {best_result.total_cost:.2f}")
        return best_solution, best_result

    def _roulette_wheel_selection(self, weights: List[float]) -> int:
        total_weight = sum(weights)
        if total_weight == 0:
            return random.randint(0, len(weights) - 1)
        pick = random.uniform(0, total_weight)
        current = 0
        for i, weight in enumerate(weights):
            current += weight
            if current > pick:
                return i
        return len(weights) - 1

    def _update_weights(self):
        for w_arr, s_arr, c_arr in [(self.weights_destroy, self.scores_destroy, self.counts_destroy),
                                    (self.weights_repair, self.scores_repair, self.counts_repair)]:
            for i in range(len(w_arr)):
                avg_score = (s_arr[i] / c_arr[i]) if c_arr[i] > 0 else 0.0
                w_arr[i] = (1 - WEIGHT_UPDATE_REACTION_FACTOR) * w_arr[i] + WEIGHT_UPDATE_REACTION_FACTOR * max(1e-6, avg_score)
                s_arr[i] = 0.0
                c_arr[i] = 0

class Constructor:
    def __init__(self, instance: Instance):
        self.inst = instance

    def initial_solution(self) -> Solution:
        customer_ids = self.inst.customer_ids.copy()
        random.shuffle(customer_ids)

        truck_routes: Dict[str, List[int]] = {f'truck_{i}': [] for i in range(NUM_TRUCKS)}

        for i, cid in enumerate(customer_ids):
            truck_index = i % NUM_TRUCKS
            truck_routes[f'truck_{truck_index}'].append(cid)

        hub_drone_assignments: Dict[int, List[int]] = {hid: [] for hid in self.inst.hub_ids}

        return Solution(
            truck_routes=truck_routes,
            hub_drone_assignments=hub_drone_assignments,
            truck_launched_assignments=[],
            unserved_requests=[]
        )

def main():
    random.seed(42)
    np.random.seed(42)

    print("--- STEP 1: Creating Random Instance ---")
    NUM_TOTAL_NODES, NUM_HUBS, NUM_CUSTOMERS = 30, 5, 24
    adjacency_matrix, node_properties = create_random_instance_data(
        num_nodes=NUM_TOTAL_NODES, num_hubs=NUM_HUBS, num_customers=NUM_CUSTOMERS,
        max_dist=200, truck_speed=TRUCK_SPEED, drone_speed=DRONE_SPEED
    )
    instance = Instance(adjacency_matrix, node_properties)

    print("\n--- STEP 2: Constructing Initial Solution ---")
    constructor = Constructor(instance)
    initial_solution = constructor.initial_solution()

    print("\n--- STEP 3: Running ALNS Metaheuristic ---")
    alns_solver = ALNS(instance)
    best_solution, best_result = alns_solver.run(initial_solution)

    print("\n" + "=" * 60)
    print(" " * 20 + "FINAL SOLUTION SUMMARY")
    print("=" * 60)
    print("\n1) Truck Routes:")
    for truck_id, route in best_solution.truck_routes.items():
        if route:
            print(f"  - {truck_id}: 0 -> {' -> '.join(map(str, route))} -> 0")
        else:
            print(f"  - {truck_id}: <unused>")

    print("\n2) Hub Drone Missions:")
    for hub_id, requests in best_solution.hub_drone_assignments.items():
        if requests:
            print(f"  - Hub {hub_id} serves: {requests}")

    print("\n3) Truck-Launched Drone Missions:")
    if best_solution.truck_launched_assignments:
        for mission in best_solution.truck_launched_assignments:
            print(f"  - Cust {mission['req_id']} from {mission['truck_id']} on leg {mission['launch']} -> {mission['rendezvous']}")
    else:
        print("  - None")

    print("\n4) Final Status:")
    if best_solution.unserved_requests:
        print(f"  - Declared Unserved: {best_solution.unserved_requests}")
    print(f"\nOverall Cost = {best_result.total_cost:.2f} "
          f"(Tardiness Penalty = {best_result.tardiness_cost:.2f}, Unserved Penalty = {best_result.unserved_cost:.2f})")
    print(f"Late Customers: {best_result.late_customer_count}, Unserved Customers: {best_result.unserved_customer_count}")
    print("=" * 60)

if __name__ == "__main__":
    main()
